In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import matplotlib.pyplot as plt

# I want to load each of the CSV files in the data directory
# and then concatenate them into a single DataFrame.

In [ ]:
import os

master_df = pd.DataFrame()
data_files = []

for filename in os.listdir("products_dataset"): # Iterates over files in the directory
    if filename.endswith(".csv"): # Check if the file ends with .csv
        data_files.append(filename) # Add the filename to the list
    else:
        continue

print(f"Found the following {len(data_files)} data files: {data_files}" )

Found the following 115 data files: ['Air Conditioners.csv', 'All Appliances.csv', 'All Car and Motorbike Products.csv', 'All Electronics.csv', 'All Exercise and Fitness.csv', 'All Grocery and Gourmet Foods.csv', 'All Home and Kitchen.csv', 'All Pet Supplies.csv', 'All Sports Fitness and Outdoors.csv', 'Amazon Fashion.csv', 'Amazon-Products.csv', 'Baby Bath Skin and Grooming.csv', 'Baby Fashion.csv', 'Baby Products.csv', 'Backpacks.csv', 'Badminton.csv', 'Bags and Luggage.csv', 'Ballerinas.csv', 'Beauty and Grooming.csv', 'Bedroom Linen.csv', 'Camera Accessories.csv', 'Cameras.csv', 'Camping and Hiking.csv', 'Car Accessories.csv', 'Car and Bike Care.csv', 'Car Electronics.csv', 'Car Parts.csv', 'Cardio Equipment.csv', 'Casual Shoes.csv', 'Clothing.csv', 'Coffee Tea and Beverages.csv', 'Cricket.csv', 'Cycling.csv', 'Diapers.csv', 'Diet and Nutrition.csv', 'Dog supplies.csv', 'Ethnic Wear.csv', 'Fashion and Silver Jewellery.csv', 'Fashion Sales and Deals.csv', 'Fashion Sandals.csv', 'Fit

# Load the dataset

In [ ]:
for i in data_files:
    file_path = os.path.join("products_dataset", i) # Construct the full file path for each file
    df = pd.read_csv(file_path) # Read the CSV file from the path into a DataFrame
    master_df = pd.concat([master_df, df], ignore_index=True) # Concatenate the DataFrame to the master DataFrame

master_df.head()

,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,Unnamed: 0
0,Lloyd 1.5 Ton 3 Star Inverter Split Ac (5 In 1...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/31UISB90sY...,https://www.amazon.in/Lloyd-Inverter-Convertib...,4.2,"2,255","₹32,999","₹58,990",NaN
1,LG 1.5 Ton 5 Star AI DUAL Inverter Split AC (C...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Convertible-Anti-Viru...,4.2,"2,948","₹46,490","₹75,990",NaN
2,LG 1 Ton 4 Star Ai Dual Inverter Split Ac (Cop...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Inverter-Convertible-...,4.2,"1,206","₹34,490","₹61,990",NaN
3,LG 1.5 Ton 3 Star AI DUAL Inverter Split AC (C...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/51JFb7FctD...,https://www.amazon.in/LG-Convertible-Anti-Viru...,4.0,69,"₹37,990","₹68,990",NaN
4,Carrier 1.5 Ton 3 Star Inverter Split AC (Copp...,appliances,Air Conditioners,https://m.media-amazon.com/images/I/41lrtqXPiW...,https://www.amazon.in/Carrier-Inverter-Split-C...,4.1,630,"₹34,490","₹67,790",NaN


# For the purposes of the task, I will be removing the category and sub-category columns
# as well as other columns whihc provide no value to the analysis.

In [ ]:
master_df = master_df.drop(columns=['main_category', 'sub_category', 'image', 'link', 'Unnamed: 0']) # Drop specified columns

master_df.head()

,name,ratings,no_of_ratings,discount_price,actual_price
0,Lloyd 1.5 Ton 3 Star Inverter Split Ac (5 In 1...,4.2,"2,255","₹32,999","₹58,990"
1,LG 1.5 Ton 5 Star AI DUAL Inverter Split AC (C...,4.2,"2,948","₹46,490","₹75,990"
2,LG 1 Ton 4 Star Ai Dual Inverter Split Ac (Cop...,4.2,"1,206","₹34,490","₹61,990"
3,LG 1.5 Ton 3 Star AI DUAL Inverter Split AC (C...,4.0,69,"₹37,990","₹68,990"
4,Carrier 1.5 Ton 3 Star Inverter Split AC (Copp...,4.1,630,"₹34,490","₹67,790"


# We want to ensure all data is empirical
# We will convert the price column to flat by removing the yen symbol using regex matching

In [ ]:
master_df['discount_price'] = master_df['discount_price'].replace('[\₹,]', '', regex=True).astype(float) # Convert discount_price to float
master_df['actual_price'] = master_df['actual_price'].replace('[\₹,]', '', regex=True).astype(float) # Convert actual_price to float

master_df['ratings'] = pd.to_numeric(master_df['ratings'], errors='coerce') # Convert ratings to float
master_df = master_df.dropna(subset=['ratings']) # Drop NaN values in ratings

master_df.head()

,name,ratings,no_of_ratings,discount_price,actual_price
0,Lloyd 1.5 Ton 3 Star Inverter Split Ac (5 In 1...,4.2,"2,255",32999.0,58990.0
1,LG 1.5 Ton 5 Star AI DUAL Inverter Split AC (C...,4.2,"2,948",46490.0,75990.0
2,LG 1 Ton 4 Star Ai Dual Inverter Split Ac (Cop...,4.2,"1,206",34490.0,61990.0
3,LG 1.5 Ton 3 Star AI DUAL Inverter Split AC (C...,4.0,69,37990.0,68990.0
4,Carrier 1.5 Ton 3 Star Inverter Split AC (Copp...,4.1,630,34490.0,67790.0


# Some derived fields can be useful for analysis.
# We will calculate the discount amount, the discount percentage, the price-rating ratio

In [ ]:
master_df['discount'] = master_df['actual_price'] - master_df['discount_price'] # Calculate discount
master_df['discount_percentage'] = ((master_df['discount'] / master_df['actual_price']) * 100).round(2) # Calculate discount percentage
master_df['price_rating_ratio'] = (master_df['actual_price'] / master_df['ratings']).round(3) # Calculate price-rating ratio

master_df.head()

,name,ratings,no_of_ratings,discount_price,actual_price,discount,discount_percentage,price_rating_ratio
0,Lloyd 1.5 Ton 3 Star Inverter Split Ac (5 In 1...,4.2,"2,255",32999.0,58990.0,25991.0,44.06,14045.238
1,LG 1.5 Ton 5 Star AI DUAL Inverter Split AC (C...,4.2,"2,948",46490.0,75990.0,29500.0,38.82,18092.857
2,LG 1 Ton 4 Star Ai Dual Inverter Split Ac (Cop...,4.2,"1,206",34490.0,61990.0,27500.0,44.36,14759.524
3,LG 1.5 Ton 3 Star AI DUAL Inverter Split AC (C...,4.0,69,37990.0,68990.0,31000.0,44.93,17247.500
4,Carrier 1.5 Ton 3 Star Inverter Split AC (Copp...,4.1,630,34490.0,67790.0,33300.0,49.12,16534.146


# The fatures operate on different scales, i need to standardise the data
# I will use Min-Max scaling to standardise the data

In [ ]:
def min_max_scaling(df, column):
    min = df[column].min() # Getting the min value in the col
    max = df[column]. max() # Getting the max value in the col

    def func(x):
        return (((x-min)/(max-min))) # Calculating min-max formula

    df[column] = df[column].apply(func) # Applying formula to all values in the column

In [10]:
scaled_df = master_df # Save my dataframe
scaled_df = scaled_df.drop(columns=['name']) # Remove non-numeric fields
scaled_df['no_of_ratings'] = scaled_df['no_of_ratings'].astype(str) # Convert object field to str
scaled_df['no_of_ratings'] = scaled_df['no_of_ratings'].replace(",", "", regex=True) # Remove commas
scaled_df['no_of_ratings'] = pd.to_numeric(scaled_df['no_of_ratings']) # Convert to numeric
scaled_df = scaled_df[scaled_df['actual_price'] <= 2000000] # Drop rows with price over 2 million as these are definite outliers


scaled_df = scaled_df.dropna() # Drop rows with missing values

scaled_df.head()

,ratings,no_of_ratings,discount_price,actual_price,discount,discount_percentage,price_rating_ratio
0,4.2,2255.0,32999.0,58990.0,25991.0,44.06,14045.238
1,4.2,2948.0,46490.0,75990.0,29500.0,38.82,18092.857
2,4.2,1206.0,34490.0,61990.0,27500.0,44.36,14759.524
3,4.0,69.0,37990.0,68990.0,31000.0,44.93,17247.500
4,4.1,630.0,34490.0,67790.0,33300.0,49.12,16534.146


In [11]:
for column in scaled_df.columns[0:]: # Iterate over each column
    min_max_scaling(scaled_df, column) # Apply min-max to each column

scaled_df.head()

,ratings,no_of_ratings,discount_price,actual_price,discount,discount_percentage,price_rating_ratio
0,0.800,0.003823,0.026393,0.036981,0.040996,0.441041,0.035220
1,0.800,0.004999,0.037186,0.047640,0.046530,0.388589,0.045371
2,0.800,0.002044,0.027586,0.038862,0.043376,0.444044,0.037011
3,0.750,0.000115,0.030386,0.043251,0.048896,0.449750,0.043251
4,0.775,0.001067,0.027586,0.042498,0.052524,0.491692,0.041462
